In [327]:
import pandas as pd
from dataclasses import dataclass
from collections import Counter
import random
import math
import numpy as np
from icecream import ic
from matplotlib import pyplot as plt
from itertools import accumulate
from itertools import combinations
from tqdm.auto import tqdm
import geopy.distance
from geopy.distance import geodesic
import networkx as nx
from collections import deque

In [328]:
cities = pd.read_csv('cities/italy.csv', header=None, names=['City', 'x', 'y'])
cities

,City,x,y
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34
5,Bolzano,46.50,11.35
6,Brescia,45.55,10.22
7,Cagliari,39.22,9.10
8,Catania,37.50,15.08
9,Ferrara,44.84,11.61


In [329]:
DIST_MATRIX = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.x, c1.y), (c2.x, c2.y)
    ).km

In [330]:
def fitness(solution):
    tot_dist=0
    for node in range(len(solution)-1):
        tot_dist -= DIST_MATRIX[solution[node], solution[node+1]]
    return tot_dist

In [331]:
def swap_mutation(solution):
    index = random.randint(1, len(solution)-3) #not the last one nor the first
    index2=index #should be higher
    while index2<=index:
        index2 = random.randint(1, len(solution)-2)
    selected_edge1 = solution[index]
    selected_edge2 = solution[index2]
    solution[index] = selected_edge2
    solution[index2] = selected_edge1
    return solution

In [332]:
@dataclass
class Individual:
    genome: list
    fitness : float = None

In [333]:
def find_closest(segments, city, visited):
    # Filtra i segmenti che contengono 'city' e hanno un'altra città non in 'visited'
    candidates = [
        (pair, distance) for pair, distance in segments
        if city in pair and (other_city := (pair - {city}).pop()) not in visited
    ]
    if len(candidates)==0:
        print("errore")
    closest_segment = min(candidates, key=lambda x: x[1])
    closest_city = (closest_segment[0] - {city}).pop()
    return (int(city), closest_city)

In [334]:
def greedy_sol(city, segments):
    solution = []
    solution.append(city)
    visited = []
    visited.append(int(city))
    while len(visited)<len(cities):
        _, c1 = find_closest(segments, city, visited)
        solution.append(c1)
        visited.append(c1)
        city=c1
    solution.append(solution[0])
    
    return solution

In [335]:
# def parent_selection(population):
#     candidates = sorted(np.random.choice(population, int(len(population)/8)), key=lambda e: e.fitness, reverse=True)
#     return candidates[0]


def inver_over_crossover(parent1, parent2):
    # Copia del primo genitore, su cui applicheremo le modifiche
    child = parent1[:]
    
    # Selezione di un punto di crossover casuale
    crossover_point = random.randint(1, len(parent1) - 2)
    
    # Segmento dal primo genitore (prima del punto di crossover)
    first_segment = parent1[:crossover_point]
    
    second_segment = []
    for city in parent2:
        if city not in first_segment:
            second_segment.append(city)
    
    # Combinazione dei segmenti per creare il figlio
    child = first_segment + second_segment
    
    return child

In [336]:
def inver_over_crossover(parent1, parent2):
    # Copia del primo genitore come base per il figlio
    child = parent1[:]
    
    # Mantieni il primo nodo identico all'ultimo
    start_node = child[0]
    end_node = child[-1]

    if start_node != end_node:
        child.append(start_node)
    
    # Iterazioni di inversioni casuali
    num_inversions = random.randint(1, len(parent1) // 2)  # Numero casuale di inversioni
    
    for _ in range(num_inversions):
        # Seleziona due punti casuali per definire l'intervallo da invertire
        i, j = sorted(random.sample(range(1, len(child) - 1), 2))
        
        # Inversione del sotto-percorso selezionato
        child[i:j + 1] = reversed(child[i:j + 1])
    
    # Assicurati che il figlio sia ciclico (chiudi il ciclo)
    if child[-1] != child[0]:
        child[-1] = child[0]
    
    return child

In [337]:
def create_random_solution():
    solution =[i for i in range(len(cities))]
    np.random.shuffle(solution)
    solution.append(solution[0])
    return solution

Actual algorithm

# Scramble mutation with strength parameter

In [338]:
def scramble_mutation(solution, strength = 0.4):
    # use the beta distribution to get a number n considering the strength:
    alpha = (1 - strength) * 5 + 1 
    beta = strength * 5 + 1
    # n between 1 and len(solution)-2
    max_n = len(solution) - 2
    n = int(1 + (max_n - 2) * random.betavariate(alpha, beta))
    indices = random.sample(range(1, len(solution)-3), n) #not the last one nor the first
    # shuffle the value of the indices:
    valori_da_mescolare = [solution[i] for i in indices]
    random.shuffle(valori_da_mescolare)
    # Riassegna i valori mescolati agli stessi indici in solution
    for i, indice in enumerate(indices):
        solution[indice] = valori_da_mescolare[i]
    return solution

### more population + selective pressure doesn't show better results: better configuration: population 20 and offspring 200.

In [339]:
def parent_selection(population):
    candidates = sorted(np.random.choice(population, int(len(population)/15)), key=lambda e: e.fitness, reverse=True)
    return candidates[0]

In [340]:
POPULATION_SIZE = 20
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)
]
population = [Individual(greedy_sol(random.randint(0, len(cities)-1), segments)) for _ in range(int(POPULATION_SIZE))]
while len(population)<POPULATION_SIZE:
    population.append(Individual(genome=create_random_solution()))
for i in population:
    i.fitness = fitness(i.genome)
population.sort(key=lambda i: i.fitness, reverse=True)
print("best solution before EA: ", population[0].fitness)
last_improvement=0
current_fitness=float('inf')*(-1)
OFFSPRING_SIZE = 200
MAX_GENERATIONS=1_000


for g in range(MAX_GENERATIONS):
    offspring = []
    
    for _ in range (OFFSPRING_SIZE):
        if np.random.random()<0.4:#mutation probability:
            p=parent_selection(population)
            o=swap_mutation(p.genome.copy())
        else:
            i1 = parent_selection(population)
            i2 = parent_selection(population)
            o = inver_over_crossover(i1.genome.copy(), i2.genome.copy())
        offspring.append(Individual(genome=o))
        
    for i in offspring:
        i.fitness = fitness(i.genome)
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    print("sol so far at generation: ", g, "is :",fitness(population[0].genome))
    if fitness(population[0].genome)>current_fitness:
        current_fitness = fitness(population[0].genome)
        last_improvement=g

population.sort(key=lambda i: i.fitness, reverse=True)
population = population[:POPULATION_SIZE]
print("best solution after EA: ", population[0].fitness)
print("final solution:", population[0])
print("last improvement:", last_improvement)

best solution before EA:  -4436.03176952516
sol so far at generation:  0 is : -4436.03176952516
sol so far at generation:  1 is : -4436.03176952516
sol so far at generation:  2 is : -4436.03176952516
sol so far at generation:  3 is : -4436.03176952516
sol so far at generation:  4 is : -4436.03176952516
sol so far at generation:  5 is : -4436.03176952516
sol so far at generation:  6 is : -4436.03176952516
sol so far at generation:  7 is : -4436.03176952516
sol so far at generation:  8 is : -4436.03176952516
sol so far at generation:  9 is : -4436.03176952516
sol so far at generation:  10 is : -4436.03176952516
sol so far at generation:  11 is : -4436.03176952516
sol so far at generation:  12 is : -4436.03176952516
sol so far at generation:  13 is : -4436.03176952516
sol so far at generation:  14 is : -4436.03176952516
sol so far at generation:  15 is : -4431.363767937219
sol so far at generation:  16 is : -4431.363767937219
sol so far at generation:  17 is : -4431.363767937219
sol so fa

#### WE CAN SAY THAT SWAP MUTATION IS THE BEST MUTATION FOR THIS ALGORITHM. I avoid to show it, but using inversion mutation the results are slightly worst than swap muation.

Also applying the simulated annealing to the solution provided by EA does not show better results!

# Simulated annealing + EA

But what if we first apply the SA and then the EA?

In [341]:
def simulated_annealing(solution):
    temperatura_iniziale = 100
    tasso_riscaldamento = 1.02
    it=0

    #one out of five approach
    miglioramenti_recenti = deque(maxlen=5) 
    miglioramenti_richiesti = 1  

    #stopping criteria:
    miglioramenti_recenti_stop = deque(maxlen=1000)
    miglioramenti_recenti_stop.append(True)

    # Initial solution: greedy one!
    x_corrente = solution
    costo_corrente = fitness(x_corrente)
    best_cost = costo_corrente
    best_sol = x_corrente

    temperatura = temperatura_iniziale
    while it<1_000:
        it+=1
        # tweak the solution
        rn = random.random()
        first_time=True
        while rn > 0.8 or first_time:
            first_time=False
            x_nuovo = swap_mutation(x_corrente.copy())
            costo_nuovo = fitness(x_nuovo)
            rn = random.random()
        
        # variation of fitness by changing sign
        delta_costo = costo_nuovo*(-1) - costo_corrente*(-1) 
        #we are sure the solution after swap mutation is valid if the previous was it.
            
        if delta_costo < 0 or (random.random() < math.exp(-delta_costo / temperatura) and delta_costo!=0):
            x_corrente = x_nuovo
            costo_corrente = costo_nuovo
            miglioramenti_recenti.append(True)
            miglioramenti_recenti_stop.append(True)
            if costo_corrente*(-1) < best_cost*(-1):
                best_cost = costo_corrente
                best_sol = x_corrente
        else:
            miglioramenti_recenti.append(False)
            miglioramenti_recenti_stop.append(False)

        if miglioramenti_recenti.count(True) > miglioramenti_richiesti:
            temperatura *= tasso_riscaldamento  # more exploration
        if miglioramenti_recenti.count(True) < miglioramenti_richiesti:
            temperatura /= tasso_riscaldamento
        
        if miglioramenti_recenti_stop.count(True)==0: #stop condition
            break
    return best_sol

In [342]:
POPULATION_SIZE = 20
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)
]
population = [Individual(simulated_annealing(greedy_sol(random.randint(0, len(cities)-1), segments))) for _ in range(int(POPULATION_SIZE))]
while len(population)<POPULATION_SIZE:
    population.append(Individual(genome=simulated_annealing(create_random_solution())))
for i in population:
    i.fitness = fitness(i.genome)
population.sort(key=lambda i: i.fitness, reverse=True)
print("best solution before EA: ", population[0].fitness)
last_improvement=0
current_fitness=float('inf')*(-1)
OFFSPRING_SIZE = 200
MAX_GENERATIONS=1_000


for g in range(MAX_GENERATIONS):
    offspring = []
    
    for _ in range (OFFSPRING_SIZE):
        if np.random.random()<0.4:#mutation probability:
            p=parent_selection(population)
            o=swap_mutation(p.genome.copy())
        else:
            i1 = parent_selection(population)
            i2 = parent_selection(population)
            o = inver_over_crossover(i1.genome.copy(), i2.genome.copy())
        offspring.append(Individual(genome=o))
        
    for i in offspring:
        i.fitness = fitness(i.genome)
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    print("sol so far at generation: ", g, "is :",fitness(population[0].genome))
    if fitness(population[0].genome)>current_fitness:
        current_fitness = fitness(population[0].genome)
        last_improvement=g

population.sort(key=lambda i: i.fitness, reverse=True)
population = population[:POPULATION_SIZE]
print("best solution after EA: ", population[0].fitness)
print("final solution:", population[0])
print("last improvement:", last_improvement)

best solution before EA:  -4498.0366734243435
sol so far at generation:  0 is : -4498.0366734243435
sol so far at generation:  1 is : -4498.0366734243435
sol so far at generation:  2 is : -4498.0366734243435
sol so far at generation:  3 is : -4498.0366734243435
sol so far at generation:  4 is : -4498.0366734243435
sol so far at generation:  5 is : -4498.0366734243435
sol so far at generation:  6 is : -4498.0366734243435
sol so far at generation:  7 is : -4498.0366734243435
sol so far at generation:  8 is : -4498.0366734243435
sol so far at generation:  9 is : -4478.490979910618
sol so far at generation:  10 is : -4432.441779776676
sol so far at generation:  11 is : -4432.441779776676
sol so far at generation:  12 is : -4432.441779776676
sol so far at generation:  13 is : -4432.441779776676
sol so far at generation:  14 is : -4432.441779776676
sol so far at generation:  15 is : -4432.441779776676
sol so far at generation:  16 is : -4432.441779776676
sol so far at generation:  17 is : -4